# Projeto final

Este projeto tem como objetivo desenvolver uma solução para o desafio [RecSys Challenge 2018](http://www.recsyschallenge.com/2018/)

Disciplina: EEL 410250 - Aprendizado de Máquina<br>
Aluno: Gustavo de Paula Santos<br>
Matrícula: 19100833<br>

In [7]:
import csv
import json
import os

import autorootcwd
import ijson
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
class million_playlist_dataset(torch.utils.data.Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx]

In [3]:
def preprocess_and_save(data, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['playlist_id', 'playlist_name', 'track_id', 'track_name', 'artist_id', 'artist_name']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for playlist in data:
            playlist_id = playlist['pid']
            playlist_name = playlist['name']
            tracks = playlist['tracks']
            for track in tracks:
                track_id = track['track_uri']
                track_name = track['track_name']
                artist_id = track['artist_uri']
                artist_name = track['artist_name']
                writer.writerow({
                    'playlist_id': playlist_id,
                    'playlist_name': playlist_name,
                    'track_id': track_id,
                    'track_name': track_name,
                    'artist_id': artist_id,
                    'artist_name': artist_name,
                })

In [4]:
def stream_json(file_path):
    with open(file_path, 'r') as file:
        objects = ijson.items(file, 'playlists.item')
        for obj in objects:
            yield obj

In [5]:
folder_path = "../spotify_million_playlist_dataset/data/"
files = os.listdir(folder_path)
files = [file for file in files if file.endswith(".json")]
files = sorted(files)

In [8]:
for file in tqdm(files):
    print(f"Processing file: {file}")
    filepath = os.path.join(folder_path, file)
    output_csv_file = os.path.join( "data/", file.replace(".json", ".csv"))
    
    data_generator = stream_json(filepath)
    preprocess_and_save(data_generator, output_csv_file)
    print(f"Done processing file: {file}")
    clear_output(wait=True)

100%|██████████| 1000/1000 [15:46<00:00,  1.06it/s]
